### 라이브러리 호출

In [1]:
import pandas_market_calendars as mcal
import pandas as pd
import warnings

warnings.filterwarnings('ignore')

### 거래소별 마켓 코드 확인

In [ ]:
'''
# Sample Market Code # 
    XKRX : Korea
    XHKG : Hongkong
    JPX  : Japan
    SSE  : Sanghai
    NYSE : NewYork
'''

calendar_name_ls = mcal.get_calendar_names()
print('########################## 거래소별 마켓 코드 확인 ##########################')
print(calendar_name_ls)
print('########################################################################\n\n\n')

### 한국 증시 캘린더

In [4]:
# 한국 증시 캘린더
market_code= 'XKRX'
calendar_kor = mcal.get_calendar(market_code) 

start_date = '2023-12-01'
end_date = '2024-01-31'

timezone= calendar_kor.tz.zone
df_stock_calendar_kor = calendar_kor.schedule(start_date=start_date, end_date=end_date,tz=timezone)
list_stock_calendar_kor = df_stock_calendar_kor.index.date.tolist()

print('########################## 한국 증시 캘린더 ##########################')
print(df_stock_calendar_kor)
print('########################################################################\n\n\n')

########################## 한국 증시 캘린더 ##########################
                         market_open               break_start  \
2023-12-01 2023-12-01 09:00:00+09:00 2023-12-01 12:00:00+09:00   
2023-12-04 2023-12-04 09:00:00+09:00 2023-12-04 12:00:00+09:00   
2023-12-05 2023-12-05 09:00:00+09:00 2023-12-05 12:00:00+09:00   
2023-12-06 2023-12-06 09:00:00+09:00 2023-12-06 12:00:00+09:00   
2023-12-07 2023-12-07 09:00:00+09:00 2023-12-07 12:00:00+09:00   
2023-12-08 2023-12-08 09:00:00+09:00 2023-12-08 12:00:00+09:00   
2023-12-11 2023-12-11 09:00:00+09:00 2023-12-11 12:00:00+09:00   
2023-12-12 2023-12-12 09:00:00+09:00 2023-12-12 12:00:00+09:00   
2023-12-13 2023-12-13 09:00:00+09:00 2023-12-13 12:00:00+09:00   
2023-12-14 2023-12-14 09:00:00+09:00 2023-12-14 12:00:00+09:00   
2023-12-15 2023-12-15 09:00:00+09:00 2023-12-15 12:00:00+09:00   
2023-12-18 2023-12-18 09:00:00+09:00 2023-12-18 12:00:00+09:00   
2023-12-19 2023-12-19 09:00:00+09:00 2023-12-19 12:00:00+09:00   
2023-12-20 2

### 미국 증시 캘린더

In [ ]:
# 한국 증시 캘린더
market_code= 'NYSE'
calendar_kor = mcal.get_calendar(market_code) 

start_date = '2023-12-01'
end_date = '2024-01-31'

timezone= calendar_kor.tz.zone
df_stock_calendar_kor = calendar_kor.schedule(start_date=start_date, end_date=end_date,tz=timezone)
list_stock_calendar_kor = df_stock_calendar_kor.index.date.tolist()

print('########################## 한국 증시 캘린더 ##########################')
print(df_stock_calendar_kor)
print('########################################################################\n\n\n')

### DB 인서트 시 Timezone, date 타입 고려하여 전처리

In [6]:
# db insert 시 dbms의 설정된 timezone으로 강제 변환될 수 있기 때문에
# date,time으로 나누어 insert 하면 안전
def get_df_market_open_time(market:str):
    calendar = mcal.get_calendar(market)
    tz= calendar.tz.zone   

    
    df = calendar.schedule(start_date=start_date, end_date=end_date,tz=tz)
    
    # 'end_date' 컬럼을 timestamp 타입으로 변환
    df['market_open'] = pd.to_datetime(df['market_open'])
    # 'market_open_date' 컬럼에는 날짜 정보만 저장 ('YYYY-MM-DD' 형식)
    df['market_open_date'] = df['market_open'].dt.strftime('%Y-%m-%d')
    # 'market_open_time' 컬럼에는 시간 정보만 저장 ('HH:mm' 형식)
    df['market_open_time'] = df['market_open'].dt.strftime('%H:%M')

    df['market_close'] = pd.to_datetime(df['market_close'])
    # 'market_open_date' 컬럼에는 날짜 정보만 저장 ('YYYY-MM-DD' 형식)
    df['market_close_date'] = df['market_close'].dt.strftime('%Y-%m-%d')
    # 'market_open_time' 컬럼에는 시간 정보만 저장 ('HH:mm' 형식)
    df['market_close_time'] = df['market_close'].dt.strftime('%H:%M')

    # 'end_date' 컬럼에서 날짜 부분만 추출하여 'date' 컬럼 추가
    df['datadate'] = df['market_open'].dt.date
    df['market_cd']=market
    df['timezone']=tz
    
    df= df.reset_index()
    
    df= df[['market_cd','datadate','timezone','market_open_date','market_open_time','market_close_date','market_close_time']]
    
    return df

df_stock_calendar_jpx = get_df_market_open_time('JPX')

print('##########################  일본 증시 캘린더 (date/time 분리) ##########################')
print(df_stock_calendar_jpx)
print('########################################################################\n\n\n')


##########################  일본 증시 캘린더 (date/time 분리) ##########################
   market_cd    datadate    timezone market_open_date market_open_time  \
0        JPX  2023-12-01  Asia/Tokyo       2023-12-01            09:00   
1        JPX  2023-12-04  Asia/Tokyo       2023-12-04            09:00   
2        JPX  2023-12-05  Asia/Tokyo       2023-12-05            09:00   
3        JPX  2023-12-06  Asia/Tokyo       2023-12-06            09:00   
4        JPX  2023-12-07  Asia/Tokyo       2023-12-07            09:00   
5        JPX  2023-12-08  Asia/Tokyo       2023-12-08            09:00   
6        JPX  2023-12-11  Asia/Tokyo       2023-12-11            09:00   
7        JPX  2023-12-12  Asia/Tokyo       2023-12-12            09:00   
8        JPX  2023-12-13  Asia/Tokyo       2023-12-13            09:00   
9        JPX  2023-12-14  Asia/Tokyo       2023-12-14            09:00   
10       JPX  2023-12-15  Asia/Tokyo       2023-12-15            09:00   
11       JPX  2023-12-18  Asia/T